In [ ]:
import pandas as pd

data = pd.read_csv('clean/data_age_filter_full_intern_wait_ts.csv', sep=';', low_memory=False)
data.tail()

In [ ]:
sorted_data = data.sort_values(by=["gravité", "date_arrivee_patient"])
sorted_data.head()

In [ ]:
sorted_data.info()

In [ ]:
print(sorted_data["gravité"].value_counts().sort_index())

In [ ]:
import matplotlib.pyplot as plt

plt.hist(data=sorted_data, x="gravité")

In [ ]:
sorted_data.iloc[259:264]

In [ ]:
sorted_data.tail()

In [ ]:
sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.iloc[259:264]

In [ ]:
columns = ["dos_id", "age", "sexe", "residence", "mode_entree", "etat_patient_entree", "motif_admission", "score_tri",
           "occupation_lits", "ratio_medecin_patient", "ratio_interne_patient", "ratio_infirmiere_patient",
           "nbre_patient_score_0", "nbre_patient_score_1", "nbre_patient_score_2", "nbre_patient_score_3",
           "nbre_patient_score_4", "nbre_patient_score_5", "nbre_patient_salle_att_ext",
           "nbre_patient_salle_att_interne", "nbre_patient_traite_sans_fin_PEC", "nbre_patient_entre_finPEC_sortie",
           "nbre_patient_UHCD", "UHCD_saturee", "est_venu_3_derniers_mois", "est_venu_derniers_48h",
           "moyenne_dynamique_temps_attente", "date_arrivee_patient", "wait_time_ext", "wait_time_interne", "wait_time"]

feature_df = pd.DataFrame(columns=columns)

In [ ]:
feature_df["dos_id"] = sorted_data["dos_id"]
feature_df["age"] = sorted_data["age"]
feature_df["sexe"] = sorted_data["sexe"]
feature_df["residence"] = sorted_data["code postal"]
feature_df["mode_entree"] = sorted_data["ven_modeEntree"]
feature_df["etat_patient_entree"] = sorted_data["ven_etat"]
feature_df["motif_admission"] = sorted_data["ven_motifAdmission"]
feature_df["score_tri"] = sorted_data["gravité"]
feature_df["nbre_patient_UHCD"] = sorted_data["nbre_pat_uhcd"]
feature_df["date_arrivee_patient"] = sorted_data["date_arrivee_patient"]

In [ ]:
feature_df.head()

In [ ]:
feature_df.columns

In [ ]:
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

In [ ]:
#Feature Transformation
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

In [ ]:
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

In [ ]:
feature_df.columns

In [ ]:
sorted_data = sorted_data.sort_values(by=["gravité", "date_arrivee_patient"])
sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)

In [ ]:
#Feature nbre_patient_score 0,1,2 calculation
from datetime import datetime, timedelta


def caluler_nbre_patient_par_score(patient_admission_ts, score, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min

    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        other_patient_admission = datetime.strptime(df_process.loc[row_index, "date_arrivee_patient"],
                                                    "%Y-%m-%d %H:%M")
        other_patient_exit = datetime.strptime(df_process.loc[row_index, "dos_fin_sortie"], "%d/%m/%Y %H:%M")
        ven_sortie = df_process.loc[row_index, "ven_sortie"]
        if (other_patient_exit - other_patient_admission) > timedelta(days=2) and not pd.isnull(ven_sortie):
            other_patient_exit = datetime.strptime(df_process.loc[row_index, "ven_sortie"][:-3], "%Y-%m-%d %H:%M:%S")

        other_patient_score = df_process.loc[row_index, "gravité"]
        other_patient_dos_id = df_process.loc[row_index, "dos_id"]

        #print("\nother_patient_admission: ", other_patient_admission)
        #print("other_patient_exit: ", other_patient_exit)
        #print("patient_admission_ts", patient_admission_ts)
        #print("patient_admission_ts <= other_patient_exit",patient_admission_ts <= other_patient_exit)

        if other_patient_score != score or patient_admission_ts < other_patient_admission:
            #print("\nother_patient_admission: ", other_patient_admission)
            #print("other_patient_exit: ", other_patient_exit)
            #print("##Break##", ", row_index: ", row_index, ", dos_id: ", dos_id, ", date admission: ", patient_admission_ts, ", counter: ", counter)
            break
        if other_patient_dos_id != dos_id and other_patient_admission <= patient_admission_ts <= other_patient_exit and other_patient_score == score:
            counter += 1
            if counter == 1:
                index = row_index
            #print("Add", ", row_index: ", row_index, ", dos_id: ", dos_id, ", other_patient_dos_id: ", other_patient_dos_id, ", date admission: ", patient_admission_ts, ", counter: ", counter)

    return counter, index

In [ ]:
feature_df = feature_df.sort_values(by=["date_arrivee_patient"])
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.columns

In [ ]:
print(sorted_data["gravité"].value_counts().sort_index())

In [ ]:
range_list = sorted_data["gravité"].value_counts().sort_index()
range_list[0]

In [ ]:
import time

start = time.time()
index = 0
borne_sup = range_list[0]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_0"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=0, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

In [ ]:
#To index dataset backup
dataset_version = 0

feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
import time

start = time.time()
index = borne_sup
borne_sup += range_list[1]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_1"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=1, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
import time

start = time.time()
index = borne_sup
borne_sup += range_list[2]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_2"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=2, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
import time

start = time.time()
index = borne_sup
borne_sup += range_list[3]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_3"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=3, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
import time

start = time.time()
index = borne_sup
borne_sup += range_list[4]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_4"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=4, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
import time

start = time.time()
index = borne_sup
borne_sup += range_list[5]
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_score_5"], index = caluler_nbre_patient_par_score(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        score=5, ranges={"min": index, "max": borne_sup}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

In [ ]:
#Feature occupation_lits caluculation
from datetime import datetime

nbre_lits_date_1 = datetime.strptime("2018-05-02", "%Y-%m-%d")
nbre_lits_date_2 = datetime.strptime("2018-11-05", "%Y-%m-%d")

#data = pd.read_csv('output/result_with_nbre_patient.csv', sep=';', encoding='latin-1')
sorted_data = sorted_data.sort_values(by=["dos_id"])
sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

In [ ]:
feature_df = feature_df.sort_values(by=["dos_id"])
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

In [ ]:
start = time.time()
for row_index in range(len(feature_df)):
    patient_admission = datetime.strptime(feature_df.loc[row_index, "date_arrivee_patient"], "%Y-%m-%d %H:%M")
    nbre_patient = sorted_data.loc[row_index, "nbre_patient"]
    if sorted_data.loc[row_index, "dos_id"] != feature_df.loc[row_index, "dos_id"]:
        print("False Dos_id matching!")
    if patient_admission < nbre_lits_date_1:
        feature_df.loc[row_index, "occupation_lits"] = nbre_patient / 367
    elif nbre_lits_date_1 <= patient_admission < nbre_lits_date_2:
        feature_df.loc[row_index, "occupation_lits"] = nbre_patient / 372
    elif patient_admission >= nbre_lits_date_2:
        feature_df.loc[row_index, "occupation_lits"] = nbre_patient / 376
    if row_index % 10000 == 0:
        print("\n*******************************************************************")
        print("row_index: ", row_index)
        print("Partial time of execution: ", time.time() - start, " s")
        print("******************************************************************\n")

end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

In [ ]:
#Feature UHCD_saturee caluculation
start = time.time()
for row_index in range(len(feature_df)):
    nbre_patient_UHCD = feature_df.loc[row_index, "nbre_patient_UHCD"]
    if nbre_patient_UHCD <= 8:
        feature_df.loc[row_index, "UHCD_saturee"] = 0
    elif nbre_patient_UHCD > 8:
        feature_df.loc[row_index, "UHCD_saturee"] = 1
    if row_index % 10000 == 0:
        print("\n*******************************************************************")
        print("row_index: ", row_index)
        print("Partial time of execution: ", time.time() - start, " s")
        print("******************************************************************\n")

end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

In [ ]:
feature_df = feature_df.astype({'UHCD_saturee': 'int32'})

In [ ]:
feature_df.head()

In [ ]:
feature_df = feature_df.sort_values(by=["date_arrivee_patient"])
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

In [ ]:
#Feature nbre_patient_salle_att_ext caluculation

from datetime import datetime

#sorted_data = pd.read_csv("output/sorted_data.csv", sep=";")
sorted_data = sorted_data.sort_values(by=["date_arrivee_patient", "TS_tri"])

sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)


def caluler_nbre_patient_salle_att_par_date(patient_admission_ts, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min
    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        other_patient_admission = datetime.strptime(df_process.loc[row_index, "date_arrivee_patient"],
                                                    "%Y-%m-%d %H:%M")
        other_patient_TS_tri = datetime.strptime(df_process.loc[row_index, "TS_tri"][:-4], "%Y-%m-%d %H:%M:%S")
        other_patient_dos_id = df_process.loc[row_index, "dos_id"]
        #print("\npatient_admission_ts: ", patient_admission_ts)
        #print("other_patient_admission: ", other_patient_admission)
        #print("other_patient_TS_tri: ", other_patient_TS_tri)
        #print("other_patient_admission *is after* patient_admission_ts ==> ", other_patient_admission > patient_admission_ts)
        #print("patient_admission_ts - other_patient_TS_tri ==> ", abs(patient_admission_ts - other_patient_TS_tri) > timedelta(days=1))
        if patient_admission_ts < other_patient_admission:
            #print(" ==> break")
            break
        if other_patient_dos_id != dos_id and other_patient_admission <= patient_admission_ts <= other_patient_TS_tri:
            counter += 1
            if counter == 1:
                index = row_index
            #print("\n## patient_admission_ts *is before* other_patient_TS_tri\n ==> counter: ", counter)

    return counter, index


In [ ]:
import time

index = 0
start = time.time()
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_salle_att_ext"], index = caluler_nbre_patient_salle_att_par_date(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        ranges={"min": index, "max": len(feature_df)}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 5000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("index: ", index)
        #print("-patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
        #print("-other_patient_ven_admission", sorted_data.loc[index, "date_arrivee_patient"])
        print("Partial time of execution: ", time.time() - start, " s")
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

In [ ]:
feature_df.nbre_patient_salle_att_ext.value_counts().sort_values()

In [ ]:
#Feature nbre_patient_salle_att_interne caluculation

from datetime import datetime

sorted_data = sorted_data.sort_values(by=["intern_wait_time_deb", "intern_wait_time_end"])
sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

In [ ]:
def caluler_nbre_patient_salle_att_interne_par_date(patient_admission_ts, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min
    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        other_patient_debut_att_interne = df_process.loc[row_index, "intern_wait_time_deb"][:-3]
        other_patient_fin_att_interne = df_process.loc[row_index, "intern_wait_time_end"][:-3]
        if not pd.isnull(other_patient_debut_att_interne) and not pd.isnull(other_patient_fin_att_interne):
            other_patient_debut_att_interne = datetime.strptime(other_patient_debut_att_interne, "%Y-%m-%d %H:%M")
            other_patient_fin_att_interne = datetime.strptime(other_patient_fin_att_interne, "%Y-%m-%d %H:%M")
            other_patient_dos_id = df_process.loc[row_index, "dos_id"]
            if patient_admission_ts < other_patient_debut_att_interne:
                #print(" ==> break")
                break
            if other_patient_dos_id != dos_id and other_patient_debut_att_interne <= patient_admission_ts <= other_patient_fin_att_interne:
                counter += 1
                if counter == 1:
                    index = row_index
        else:
            print("\nother_patient_debut_att_interne: ", other_patient_debut_att_interne)
            print("other_patient_fin_att_interne: ", other_patient_fin_att_interne)

    return counter, index

In [ ]:
import time

index = 0
start = time.time()
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_salle_att_interne"], index = caluler_nbre_patient_salle_att_interne_par_date(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        ranges={"min": index, "max": len(feature_df)}, dos_id=feature_df.loc[row_i, "dos_id"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

In [ ]:
feature_df.nbre_patient_salle_att_interne.value_counts().sort_values()

In [ ]:
#Feature nbre_patient_traites_sans_fin_PEC caluculation

from datetime import datetime
import numpy as np

#sorted_data = None
#sorted_data = pd.read_csv("output/sorted_data.csv", sep=";")
#sorted_data = sorted_data.astype({'h_debut_att': 'str', 'h_fin_att': 'str'})
"""sorted_data = pd.read_csv("data/0_Extration_Urgence.csv", sep=";", encoding="latin-1")
"""
#sorted_data = sorted_data.astype({'h_debut_att': 'str', 'h_fin_att': 'str'})
#sorted_data["h_fin_box"].replace(' ', np.nan)
sorted_data['h_fin_box'].mask(sorted_data['h_fin_box'] == ' ', np.nan, inplace=True)
sorted_data['dos_fin_PEC'].mask(sorted_data['dos_fin_PEC'] == ' ', np.nan, inplace=True)
sorted_data["h_fin_box"] = pd.to_datetime(sorted_data["h_fin_box"], format="%d/%m/%Y %H:%M").dt.strftime(
    "%Y-%m-%d %H:%M")
#sorted_data["h_fin_att"] = pd.to_datetime(sorted_data["h_fin_att"], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d %H:%M")
#sorted_data["h_fin_att"] = pd.to_datetime(sorted_data["h_fin_att"], infer_datetime_format=True, errors="coerce")

sorted_data = sorted_data.sort_values(by=["h_fin_box", "dos_fin_PEC"])

sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

In [ ]:
def caluler_nbre_patient_traites_sans_fin_pec_par_date(patient_admission_ts, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min
    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        #print(df_process.loc[row_index, "h_debut_att"])
        other_patient_fin_box = df_process.loc[row_index, "h_fin_box"]
        other_patient_fin_pec = df_process.loc[row_index, "dos_fin_PEC"]

        if not pd.isnull(other_patient_fin_box) and not pd.isnull(other_patient_fin_pec):
            other_patient_fin_box = datetime.strptime(other_patient_fin_box, "%Y-%m-%d %H:%M")
            other_patient_fin_pec = datetime.strptime(other_patient_fin_pec, "%Y-%m-%d %H:%M")
            other_patient_dos_id = df_process.loc[row_index, "dos_id"]
            #print("\npatient_admission_ts: ", patient_admission_ts)
            #print("other_patient_admission: ", other_patient_admission)
            #print("other_patient_TS_tri: ", other_patient_TS_tri)
            #print("other_patient_admission *is after* patient_admission_ts ==> ", other_patient_admission > patient_admission_ts)
            #print("patient_admission_ts - other_patient_TS_tri ==> ", abs(patient_admission_ts - other_patient_TS_tri) > timedelta(days=1))
            if patient_admission_ts < other_patient_fin_box:
                #print(" ==> break")
                break
            if other_patient_dos_id != dos_id and other_patient_fin_box <= patient_admission_ts <= other_patient_fin_pec:
                counter += 1
                if counter == 1:
                    index = row_index
                    """print("\n.......................................................................")
                    print("patient_ven_admission", patient_admission_ts)
                    print(index)
                    print("other_patient_debut_attente", other_patient_debut_att_interne)
                    print("other_patient_fin_attente", other_patient_fin_att_interne)
                    print(".......................................................................\n")"""
                #print("\n## patient_admission_ts *is before* other_patient_TS_tri\n ==> counter: ", counter)
        else:
            print("\nother_patient_fin_box: ", other_patient_fin_box)
            print("other_patient_fin_pec: ", other_patient_fin_pec)

    return counter, index

In [ ]:
feature_df.columns

In [ ]:
import time

index = 0
start = time.time()
for row_i in range(len(feature_df)):
    feature_df.loc[
        row_i, "nbre_patient_traite_sans_fin_PEC"], index = caluler_nbre_patient_traites_sans_fin_pec_par_date(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        ranges={"min": index, "max": len(feature_df)}, dos_id=feature_df.loc[row_i, "dos_id"])
    """print("\n###################################################################")
    print("patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    print(index)
    print("other_patient_debut_attente",sorted_data.loc[row_i, "h_debut_att"])
    print("other_patient_fin_attente",sorted_data.loc[row_i, "h_fin_att"])
    print("###################################################################\n")"""
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

#feature_df.head()

In [ ]:
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

In [ ]:
#Feature nbre_patient_entre_finPEC_sortie caluculation

from datetime import datetime
import numpy as np

#sorted_data = None
#sorted_data = pd.read_csv("output/sorted_data.csv", sep=";")
#sorted_data = sorted_data.astype({'h_debut_att': 'str', 'h_fin_att': 'str'})

"""sorted_data = pd.read_csv("data/0_Extration_Urgence.csv", sep=";", encoding="latin-1")
"""
#sorted_data = sorted_data.astype({'h_debut_att': 'str', 'h_fin_att': 'str'})
#sorted_data["h_fin_box"].replace(' ', np.nan)
sorted_data['dos_fin_PEC'].mask(sorted_data['dos_fin_PEC'] == ' ', np.nan, inplace=True)
sorted_data['dos_fin_sortie'].mask(sorted_data['dos_fin_sortie'] == ' ', np.nan, inplace=True)
#sorted_data["dos_fin_PEC"] = pd.to_datetime(sorted_data["dos_fin_PEC"], format="%d/%m/%Y %H:%M").dt.strftime(
#    "%Y-%m-%d %H:%M")
#sorted_data["h_fin_att"] = pd.to_datetime(sorted_data["h_fin_att"], format="%d/%m/%Y %H:%M").dt.strftime("%Y-%m-%d %H:%M")
#sorted_data["h_fin_att"] = pd.to_datetime(sorted_data["h_fin_att"], infer_datetime_format=True, errors="coerce")

sorted_data = sorted_data.sort_values(by=["dos_fin_PEC", "dos_fin_sortie"])

sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

In [ ]:
def caluler_nbre_patient_entre_fin_pec_sortie_par_date(patient_admission_ts, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min
    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        #print(df_process.loc[row_index, "h_debut_att"])
        other_patient_fin_pec = df_process.loc[row_index, "dos_fin_PEC"]
        other_patient_fin_sortie = df_process.loc[row_index, "dos_fin_sortie"]

        if not pd.isnull(other_patient_fin_pec) and not pd.isnull(other_patient_fin_sortie):
            other_patient_fin_pec = datetime.strptime(other_patient_fin_pec, "%Y-%m-%d %H:%M")
            other_patient_fin_sortie = datetime.strptime(other_patient_fin_sortie, "%d/%m/%Y %H:%M")
            other_patient_dos_id = df_process.loc[row_index, "dos_id"]
            ven_sortie = df_process.loc[row_index, "ven_sortie"]
            #print("\npatient_admission_ts: ", patient_admission_ts)
            #print("other_patient_admission: ", other_patient_admission)
            #print("other_patient_TS_tri: ", other_patient_TS_tri)
            #print("other_patient_admission *is after* patient_admission_ts ==> ", other_patient_admission > patient_admission_ts)
            #print("patient_admission_ts - other_patient_TS_tri ==> ", abs(patient_admission_ts - other_patient_TS_tri) > timedelta(days=1))
            if (other_patient_fin_sortie - other_patient_fin_pec) > timedelta(days=2) and not pd.isnull(ven_sortie):
                #print(ven_sortie)
                other_patient_fin_sortie = datetime.strptime(df_process.loc[row_index, "ven_sortie"][:-3],
                                                             "%Y-%m-%d %H:%M:%S")
            if patient_admission_ts < other_patient_fin_pec:
                #print(" ==> break")
                break
            if other_patient_dos_id != dos_id and other_patient_fin_pec <= patient_admission_ts <= other_patient_fin_sortie:
                counter += 1
                if counter == 1:
                    index = row_index
                    """print("\n.......................................................................")
                    print("patient_ven_admission", patient_admission_ts)
                    print(index)
                    print("other_patient_fin_pec", other_patient_fin_pec)
                    print("other_patient_fin_sortie", other_patient_fin_sortie)
                    print(".......................................................................\n")"""
                #print("\n## patient_admission_ts *is before* other_patient_TS_tri\n ==> counter: ", counter)
        else:
            print("\nother_patient_fin_pec: ", other_patient_fin_pec)
            print("other_patient_fin_sortie: ", other_patient_fin_sortie)

    return counter, index

In [59]:
import time

index = 0
start = time.time()
for row_i in range(len(feature_df)):
    feature_df.loc[
        row_i, "nbre_patient_entre_finPEC_sortie"], index = caluler_nbre_patient_entre_fin_pec_sortie_par_date(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        ranges={"min": index, "max": len(feature_df)}, dos_id=feature_df.loc[row_i, "dos_id"])
    """print("\n###################################################################")
    print("patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    print(index)
    print("other_patient_debut_attente",sorted_data.loc[index, "h_debut_att"])
    print("other_patient_fin_attente",sorted_data.loc[index, "h_fin_att"])
    print("###################################################################\n")"""
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        """print("patient_ven_admission: ", feature_df.loc[row_i, "date_arrivee_patient"])
        print("\nother_patient_fin_pec: ", sorted_data.loc[index, "dos_fin_PEC"])
        print("other_patient_fin_sortie: ", sorted_data.loc[index, "dos_fin_sortie"])"""
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

#feature_df.head()


*******************************************************************
row_i:  0
Partial time of execution:  0.0009992122650146484  s
index:  0
******************************************************************


*******************************************************************
row_i:  10000
Partial time of execution:  8.832453966140747  s
index:  9990
******************************************************************


*******************************************************************
row_i:  20000
Partial time of execution:  17.958512544631958  s
index:  19975
******************************************************************


*******************************************************************
row_i:  30000
Partial time of execution:  27.147369146347046  s
index:  29964
******************************************************************


*******************************************************************
row_i:  40000
Partial time of execution:  35.468557357788086  s
index:  39970


In [60]:
dataset_version

12

In [61]:
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,ratio_medecin_patient,...,nbre_patient_entre_finPEC_sortie,nbre_patient_UHCD,UHCD_saturee,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,NaN,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 08:46,NaN,NaN,NaN
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,NaN,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:00,NaN,NaN,NaN
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,NaN,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:06,NaN,NaN,NaN
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,NaN,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:15,NaN,NaN,NaN
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,NaN,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:20,NaN,NaN,NaN


In [62]:
#Feature nbre_patient_urgence caluculation

import numpy as np

sorted_data["dos_fin_sortie"] = pd.to_datetime(sorted_data["dos_fin_sortie"], format="%d/%m/%Y %H:%M").dt.strftime(
    "%Y-%m-%d %H:%M")

sorted_data = sorted_data.sort_values(by=["date_arrivee_patient", "dos_fin_sortie"])

sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data.head()

,dos_id,age,sexe,code postal,nbre_patient,ven_etat,ven_motifAdmission,gravité,ven_modeEntree,ven_modeSortie,...,mode_sortie_id,dest,ven_admission,ven_sortie,ven_date_sortie_reelle,nbre_pat_uhcd,valid_ts_tri,intern_wait_time_calculable,intern_wait_time_deb,intern_wait_time_end
0,2226,46,2,81290.0,335,2,Non precise,4,11.0,64.0,...,3,2701.0,2019-01-22 08:46:00.00,2019-01-23 13:00:00.00,2019-01-23 13:00:00.00,11,1.0,1.0,2019-01-22 09:13:00,2019-01-22 09:13:00
1,2228,16,1,81100.0,340,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:00:00.00,2019-01-22 10:06:00.00,2019-01-22 10:06:00.00,11,1.0,1.0,2019-01-22 09:29:00,2019-01-22 09:29:00
2,2229,63,2,81700.0,341,2,Non precise,5,11.0,64.0,...,3,2701.0,2019-01-22 09:06:00.00,2019-01-24 11:27:00.00,2019-01-24 11:27:00.00,11,1.0,1.0,2019-01-22 09:31:00,2019-01-22 10:08:00
3,2230,38,1,81100.0,341,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:15:00.00,2019-01-22 11:29:00.00,2019-01-22 11:29:00.00,11,1.0,1.0,2019-01-22 09:40:00,2019-01-22 10:09:00
4,2231,55,2,81290.0,342,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:20:00.00,2019-01-22 10:45:00.00,2019-01-22 10:45:00.00,11,1.0,1.0,2019-01-22 09:43:00,2019-01-22 10:10:00


In [63]:
from datetime import datetime

feature_df = feature_df.sort_values(by=["date_arrivee_patient"])
feature_df.insert(9, "nbre_patient_urgence", np.nan, True)
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)

In [64]:
def caluler_nbre_patient_urgence_par_date(patient_admission_ts, ranges, dos_id):
    counter = 0
    range_min = ranges["min"]
    range_max = ranges["max"]
    index = range_min

    df_process = sorted_data.iloc[range_min:range_max]
    for row_index in range(range_min, range_max):
        #print("df_process.loc[row_index, 'date_arrivee_patient']: ", df_process.loc[row_index, "date_arrivee_patient"])
        #print("df_process.loc[row_index, 'dos_fin_sortie']: ", df_process.loc[row_index, "dos_fin_sortie"])
        #print("patient_admission_ts: ", patient_admission_ts)

        other_patient_admission = datetime.strptime(df_process.loc[row_index, "date_arrivee_patient"],
                                                    "%Y-%m-%d %H:%M")
        other_patient_exit = datetime.strptime(df_process.loc[row_index, "dos_fin_sortie"], "%Y-%m-%d %H:%M")
        ven_sortie = df_process.loc[row_index, "ven_sortie"]

        if (other_patient_exit - other_patient_admission) > timedelta(days=2) and not pd.isnull(ven_sortie):
            other_patient_exit = datetime.strptime(ven_sortie[:-3], "%Y-%m-%d %H:%M:%S")

        other_patient_dos_id = df_process.loc[row_index, "dos_id"]

        if patient_admission_ts < other_patient_admission:
            break
        if other_patient_dos_id != dos_id and other_patient_admission <= patient_admission_ts <= other_patient_exit:
            counter += 1
            if counter == 1:
                index = row_index
            #print("Add", ", row_index: ", row_index, ", dos_id: ", dos_id, ", other_patient_dos_id: ", other_patient_dos_id, ", date admission: ", patient_admission_ts, ", counter: ", counter)

    return counter, index

In [65]:
import time

start = time.time()
index = 0
for row_i in range(len(feature_df)):
    feature_df.loc[row_i, "nbre_patient_urgence"], index = caluler_nbre_patient_urgence_par_date(
        patient_admission_ts=datetime.strptime(feature_df.loc[row_i, "date_arrivee_patient"], "%Y-%m-%d %H:%M"),
        ranges={"min": index, "max": len(feature_df)}, dos_id=feature_df.loc[row_i, "dos_id"])
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("index: ", index)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()


*******************************************************************
row_i:  0
Partial time of execution:  0.002004384994506836  s
index:  0
******************************************************************


*******************************************************************
row_i:  10000
Partial time of execution:  27.5061137676239  s
index:  9998
******************************************************************


*******************************************************************
row_i:  20000
Partial time of execution:  51.548943519592285  s
index:  19969
******************************************************************


*******************************************************************
row_i:  30000
Partial time of execution:  76.70356726646423  s
index:  29992
******************************************************************


*******************************************************************
row_i:  40000
Partial time of execution:  97.51524448394775  s
index:  39958
****

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,nbre_patient_entre_finPEC_sortie,nbre_patient_UHCD,UHCD_saturee,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 08:46,NaN,NaN,NaN
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:00,NaN,NaN,NaN
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:06,NaN,NaN,NaN
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:15,NaN,NaN,NaN
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:20,NaN,NaN,NaN


In [66]:
feature_df.nbre_patient_urgence.value_counts().sort_index()

0.0      312
1.0      934
2.0     1586
3.0     2044
4.0     2380
5.0     2515
6.0     2701
7.0     2922
8.0     3039
9.0     3249
10.0    3554
11.0    3774
12.0    3876
13.0    4059
14.0    4283
15.0    4503
16.0    4486
17.0    4345
18.0    4200
19.0    4013
20.0    3512
21.0    3254
22.0    2998
23.0    2719
24.0    2396
25.0    2032
26.0    1650
27.0    1362
28.0    1173
29.0     922
30.0     737
31.0     637
32.0     513
33.0     381
34.0     263
35.0     193
36.0     166
37.0     163
38.0     107
39.0      73
40.0      45
41.0      33
42.0      31
43.0      18
44.0      13
Name: nbre_patient_urgence, dtype: int64

In [67]:
#Feature occupation_salles_att
feature_df.insert(10, "occupation_salles_att", np.nan, True)
feature_df["occupation_salles_att"] = (feature_df["nbre_patient_salle_att_ext"] + feature_df["nbre_patient_salle_att_interne"]) / feature_df["nbre_patient_urgence"]

In [68]:
feature_df["occupation_salles_att"].fillna(0, inplace=True)
feature_df["occupation_salles_att"].replace([np.inf], 0, inplace=True)

In [69]:
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,nbre_patient_entre_finPEC_sortie,nbre_patient_UHCD,UHCD_saturee,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 08:46,NaN,NaN,NaN
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:00,NaN,NaN,NaN
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:06,NaN,NaN,NaN
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:15,NaN,NaN,NaN
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:20,NaN,NaN,NaN


In [70]:
feature_df["occupation_salles_att"].value_counts().sort_index()

0.000000    9172
0.041667       1
0.045455       4
0.047619       4
0.050000       8
            ... 
3.250000       1
3.333333       1
4.333333       1
4.600000       1
5.500000       1
Name: occupation_salles_att, Length: 407, dtype: int64

In [71]:
#Features wait_time (s) caluculation

import numpy as np
import pandas as pd
from datetime import datetime


In [72]:
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)
sorted_data = sorted_data.sort_values(by=["dos_id"])
sorted_data.to_csv("output/sorted_data.csv", sep=";", index=False)
sorted_data = pd.read_csv("output/sorted_data.csv", sep=";", low_memory=False)

In [73]:
sorted_data.head()

,dos_id,age,sexe,code postal,nbre_patient,ven_etat,ven_motifAdmission,gravité,ven_modeEntree,ven_modeSortie,...,mode_sortie_id,dest,ven_admission,ven_sortie,ven_date_sortie_reelle,nbre_pat_uhcd,valid_ts_tri,intern_wait_time_calculable,intern_wait_time_deb,intern_wait_time_end
0,2226,46,2,81290.0,335,2,Non precise,4,11.0,64.0,...,3,2701.0,2019-01-22 08:46:00.00,2019-01-23 13:00:00.00,2019-01-23 13:00:00.00,11,1.0,1.0,2019-01-22 09:13:00,2019-01-22 09:13:00
1,2228,16,1,81100.0,340,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:00:00.00,2019-01-22 10:06:00.00,2019-01-22 10:06:00.00,11,1.0,1.0,2019-01-22 09:29:00,2019-01-22 09:29:00
2,2229,63,2,81700.0,341,2,Non precise,5,11.0,64.0,...,3,2701.0,2019-01-22 09:06:00.00,2019-01-24 11:27:00.00,2019-01-24 11:27:00.00,11,1.0,1.0,2019-01-22 09:31:00,2019-01-22 10:08:00
3,2230,38,1,81100.0,341,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:15:00.00,2019-01-22 11:29:00.00,2019-01-22 11:29:00.00,11,1.0,1.0,2019-01-22 09:40:00,2019-01-22 10:09:00
4,2231,55,2,81290.0,342,2,Non precise,5,11.0,64.0,...,1,NaN,2019-01-22 09:20:00.00,2019-01-22 10:45:00.00,2019-01-22 10:45:00.00,11,1.0,1.0,2019-01-22 09:43:00,2019-01-22 10:10:00


In [74]:
sorted_data["date_arrivee_patient"].describe()

count                88166
unique               87650
top       2021-04-08 18:09
freq                     3
Name: date_arrivee_patient, dtype: object

In [75]:
sorted_data["date_arrivee_patient"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 88166 entries, 0 to 88165
Series name: date_arrivee_patient
Non-Null Count  Dtype 
--------------  ----- 
88166 non-null  object
dtypes: object(1)
memory usage: 688.9+ KB


In [76]:
#date_arrivee_patient = datetime.strptime(sorted_data.loc[0, "date_arrivee_patient"], "%Y-%m-%d %H:%M")

In [77]:
feature_df = feature_df.sort_values(by=["dos_id"])
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)
feature_df = pd.read_csv("output/feature_df.csv", sep=";", low_memory=False)
feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,nbre_patient_entre_finPEC_sortie,nbre_patient_UHCD,UHCD_saturee,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 08:46,NaN,NaN,NaN
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:00,NaN,NaN,NaN
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:06,NaN,NaN,NaN
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:15,NaN,NaN,NaN
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,0.0,11,1,NaN,NaN,NaN,2019-01-22 09:20,NaN,NaN,NaN


In [78]:
from datetime import datetime, timedelta


def calulate_wait_times_per_date(row, dos_id, moy_dyn_time):
    date_arrivee_patient = datetime.strptime(sorted_data.loc[row, "date_arrivee_patient"], "%Y-%m-%d %H:%M")
    ts_tri = datetime.strptime(sorted_data.loc[row, "TS_tri"][:-4], "%Y-%m-%d %H:%M:%S")

    wait_time_interne_deb = datetime.strptime(sorted_data.loc[row, "intern_wait_time_deb"][:-3], "%Y-%m-%d %H:%M")
    wait_time_interne_end = datetime.strptime(sorted_data.loc[row, "intern_wait_time_end"][:-3], "%Y-%m-%d %H:%M")

    fin_loc_iao = sorted_data.loc[row, 'fin_loc_iao']
    h_debut_att = sorted_data.loc[row, 'h_debut_att']

    if wait_time_interne_deb > wait_time_interne_end:
        if not pd.isnull(h_debut_att):
            wait_time_interne_deb = datetime.strptime(h_debut_att, "%Y-%m-%d %H:%M")
        elif not pd.isnull(fin_loc_iao):
            wait_time_interne_deb = datetime.strptime(fin_loc_iao, "%Y-%m-%d %H:%M")
    if wait_time_interne_deb > wait_time_interne_end:
        wait_time_interne_deb = wait_time_interne_end

    if dos_id == sorted_data.loc[row, 'dos_id']:
        wait_time_ext = (ts_tri - date_arrivee_patient).total_seconds() // 60
        wait_time_interne = (wait_time_interne_end - wait_time_interne_deb).total_seconds() // 60

        feature_df.loc[row, "wait_time_ext"] = wait_time_ext
        feature_df.loc[row, "wait_time_interne"] = wait_time_interne

        wait_time = wait_time_ext + wait_time_interne
        feature_df.loc[row, "wait_time"] = wait_time

        #feature moyenne_dynamique_temps_attente calculation
        if len(moy_dyn_time) < 15:
            moy_dyn_time.append(int(wait_time))
            feature_df.loc[row, "moyenne_dynamique_temps_attente"] = np.mean(moy_dyn_time)
        else:
            moy_dyn_time = moy_dyn_time[1:]
            moy_dyn_time.append(int(wait_time))
            feature_df.loc[row, "moyenne_dynamique_temps_attente"] = np.mean(moy_dyn_time)

    else:
        print("dos_id order not valid!")

    return moy_dyn_time

In [79]:
import time

start = time.time()
moy_dyn_time_list = []
for row_i in range(len(feature_df)):
    moy_dyn_time_list = calulate_wait_times_per_date(row=row_i, dos_id=feature_df.loc[row_i, "dos_id"],
                                                     moy_dyn_time=moy_dyn_time_list)
    #print("\n patient_ven_admission", feature_df.loc[row_i, "date_arrivee_patient"])
    #print(index)
    #print("other_patient_ven_admission",sorted_data.loc[index, "date_arrivee_patient"])
    if row_i % 10000 == 0:
        print("\n*******************************************************************")
        print("row_i: ", row_i)
        print("Partial time of execution: ", time.time() - start, " s")
        print("moy_dyn_time_list: ", moy_dyn_time_list)
        print("******************************************************************\n")
end = time.time()
print("The time of execution of above program is :", end - start, " s")

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()


*******************************************************************
row_i:  0
Partial time of execution:  0.0030057430267333984  s
moy_dyn_time_list:  [27]
******************************************************************


*******************************************************************
row_i:  10000
Partial time of execution:  7.547046661376953  s
moy_dyn_time_list:  [29, 75, 69, 27, 110, 81, 72, 82, 16, 12, 11, 10, 5, 7, 5]
******************************************************************


*******************************************************************
row_i:  20000
Partial time of execution:  14.23062801361084  s
moy_dyn_time_list:  [141, 170, 145, 170, 9, 291, 147, 16, 132, 151, 160, 118, 71, 138, 121]
******************************************************************


*******************************************************************
row_i:  30000
Partial time of execution:  19.247760772705078  s
moy_dyn_time_list:  [364, 29, 116, 221, 11, 8, 6, 3, 9, 4, 8, 69, 4, 80

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,nbre_patient_entre_finPEC_sortie,nbre_patient_UHCD,UHCD_saturee,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,0.0,11,1,NaN,NaN,27.000000,2019-01-22 08:46,27.0,0.0,27.0
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,0.0,11,1,NaN,NaN,28.000000,2019-01-22 09:00,29.0,0.0,29.0
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,0.0,11,1,NaN,NaN,39.333333,2019-01-22 09:06,25.0,37.0,62.0
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,0.0,11,1,NaN,NaN,43.000000,2019-01-22 09:15,25.0,29.0,54.0
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,0.0,11,1,NaN,NaN,44.400000,2019-01-22 09:20,23.0,27.0,50.0


In [80]:
feature_df.shape

(88166, 33)

In [81]:
#Temporal features (day of the week, ...)
feature_df.insert(26, "quarter", np.nan, True)
feature_df.insert(26, "month", np.nan, True)
feature_df.insert(26, "week", np.nan, True)
feature_df.insert(26, "is_weekend", np.nan, True)
feature_df.insert(26, "day_of_week", np.nan, True)

In [82]:
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,week,month,quarter,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,NaN,NaN,NaN,NaN,NaN,27.000000,2019-01-22 08:46,27.0,0.0,27.0
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,NaN,NaN,NaN,NaN,NaN,28.000000,2019-01-22 09:00,29.0,0.0,29.0
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,NaN,NaN,NaN,NaN,NaN,39.333333,2019-01-22 09:06,25.0,37.0,62.0
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,NaN,NaN,NaN,NaN,NaN,43.000000,2019-01-22 09:15,25.0,29.0,54.0
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,NaN,NaN,NaN,NaN,NaN,44.400000,2019-01-22 09:20,23.0,27.0,50.0


In [83]:
feature_df['date_arrivee_patient'] = pd.to_datetime(feature_df['date_arrivee_patient'])

In [84]:
feature_df["quarter"] = feature_df["date_arrivee_patient"].dt.quarter
feature_df["month"] = feature_df["date_arrivee_patient"].dt.month
feature_df["week"] = feature_df["date_arrivee_patient"].dt.isocalendar().week
feature_df["day_of_week"] = feature_df["date_arrivee_patient"].dt.weekday
feature_df["is_weekend"] = feature_df["day_of_week"] >= 5

feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_admission,score_tri,occupation_lits,nbre_patient_urgence,...,week,month,quarter,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,Non precise,4,0.890957,0.0,...,4,1,1,NaN,NaN,27.000000,2019-01-22 08:46:00,27.0,0.0,27.0
1,2228,16,1,81100.0,11.0,2,Non precise,5,0.904255,1.0,...,4,1,1,NaN,NaN,28.000000,2019-01-22 09:00:00,29.0,0.0,29.0
2,2229,63,2,81700.0,11.0,2,Non precise,5,0.906915,2.0,...,4,1,1,NaN,NaN,39.333333,2019-01-22 09:06:00,25.0,37.0,62.0
3,2230,38,1,81100.0,11.0,2,Non precise,5,0.906915,3.0,...,4,1,1,NaN,NaN,43.000000,2019-01-22 09:15:00,25.0,29.0,54.0
4,2231,55,2,81290.0,11.0,2,Non precise,5,0.909574,4.0,...,4,1,1,NaN,NaN,44.400000,2019-01-22 09:20:00,23.0,27.0,50.0


In [85]:
#Add motif_code to feature_df
feature_df.insert(6, "motif_code", np.nan, True)
feature_df = feature_df.sort_values(by=["dos_id"])
sorted_data = sorted_data.sort_values(by=["dos_id"])

feature_df["motif_code"] = sorted_data["motif_code"]

In [ ]:
feature_df.insert(26, "hour_of_day", np.nan, True)
feature_df['date_arrivee_patient'] = pd.to_datetime(feature_df['date_arrivee_patient'])
feature_df['hour_of_day'] = feature_df['date_arrivee_patient'].dt.hour

In [86]:
feature_df.to_csv("output/feature_df.csv", sep=";", index=False)

#To create dataset backup
feature_df.to_csv("output/feature_df_bak_" + str(dataset_version) + ".csv", sep=";", index=False)
dataset_version += 1

feature_df.head()

,dos_id,age,sexe,residence,mode_entree,etat_patient_entree,motif_code,motif_admission,score_tri,occupation_lits,...,week,month,quarter,est_venu_3_derniers_mois,est_venu_derniers_48h,moyenne_dynamique_temps_attente,date_arrivee_patient,wait_time_ext,wait_time_interne,wait_time
0,2226,46,2,81290.0,11.0,2,R52.9,Non precise,4,0.890957,...,4,1,1,NaN,NaN,27.000000,2019-01-22 08:46:00,27.0,0.0,27.0
1,2228,16,1,81100.0,11.0,2,M79.61,Non precise,5,0.904255,...,4,1,1,NaN,NaN,28.000000,2019-01-22 09:00:00,29.0,0.0,29.0
2,2229,63,2,81700.0,11.0,2,J02.9,Non precise,5,0.906915,...,4,1,1,NaN,NaN,39.333333,2019-01-22 09:06:00,25.0,37.0,62.0
3,2230,38,1,81100.0,11.0,2,R22.1,Non precise,5,0.906915,...,4,1,1,NaN,NaN,43.000000,2019-01-22 09:15:00,25.0,29.0,54.0
4,2231,55,2,81290.0,11.0,2,T13.9,Non precise,5,0.909574,...,4,1,1,NaN,NaN,44.400000,2019-01-22 09:20:00,23.0,27.0,50.0
